In [1]:
import numpy
import tensorflow as tf
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
seed = 7
numpy.random.seed(seed)

In [3]:
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values

In [4]:
print(dataset)

[[0.02 0.0371 0.0428 ... 0.009 0.0032 'R']
 [0.0453 0.0523 0.0843 ... 0.0052 0.0044 'R']
 [0.0262 0.0582 0.1099 ... 0.0095 0.0078 'R']
 ...
 [0.0522 0.0437 0.018 ... 0.0077 0.0031 'M']
 [0.0303 0.0353 0.049 ... 0.0036 0.0048 'M']
 [0.026 0.0363 0.0136 ... 0.0061 0.0115 'M']]


In [5]:
print(dataframe)

         0       1       2       3       4       5       6       7       8   \
0    0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
203  0.0187  0.0346  0.0168  0.0177  0.0393  0.1630  0.2028  0.1694  0.2328   
204  0.0323  0.0101  0.0298  0.0564  0.0760  0.0958  0.0990  0.1018  0.1030   
205  0.0522  0.0437  0.0180  0.0292  0.0351  0.1171  0.1257  0.1178  0.1258   
206  0.0303  0.0353  0.0490  0.0608  0.0167  0.1354  0.1465  0.1123  0.1945   
207  0.0260  0.0363  0.0136  0.0272  0.0214  0.0338  0.0655  0.1400  0.1843   

         9   ...      51      52      53      54   

In [6]:
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [7]:
print(X[0].shape)
print(X.shape)

(60,)
(208, 60)


In [8]:
print(Y.shape)

(208,)


In [9]:
encoder_Y = preprocessing.LabelEncoder()
encoder_Y.fit(Y)
LabelEncoder()

LabelEncoder()

In [10]:
print(encoder_Y.classes_)

['M' 'R']


In [11]:
encoder_Y.transform(Y)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
def create_baseline():
    model = Sequential()
    model.add(Dense(60, activation="relu",    kernel_initializer="normal", input_dim=60))
    model.add(Dense(60, activation="relu",    kernel_initializer="normal"))
    model.add(Dense(1,  activation="sigmoid", kernel_initializer="normal"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [24]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 81.68% (6.09%)


In [26]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 83.64% (4.90%)


In [27]:
def create_smaller():
    model = Sequential()
    model.add(Dense(60, activation="relu",    kernel_initializer="normal"))
    model.add(Dense(30, activation="relu",    kernel_initializer="normal"))
    model.add(Dense(1,  activation="sigmoid", kernel_initializer="normal"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [28]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.04% (4.18%)


In [12]:
def create_larger():
    model = Sequential()
    model.add(Dense(60, activation="relu",    kernel_initializer="normal", input_dim=60))
    model.add(Dense(60, activation="relu",    kernel_initializer="normal"))
    model.add(Dense(30, activation="relu",    kernel_initializer="normal"))
    model.add(Dense(1,  activation="sigmoid", kernel_initializer="normal"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [14]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

larger: 85.54% (3.84%)


In [23]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60,  activation="relu",   kernel_initializer="normal")(inputs)
    outputs= Dense(1 , activation="sigmoid", kernel_initializer="normal")(layer1) 
    
    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [24]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=functional_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Functional Api: Accuracy %.2f%%, Error (%.2f%%)" % (results.mean()*100, results.std()*100))

Functional Api: Accuracy 85.59%, Error (2.96%)


In [25]:
def checking_overfiting():
    model = Sequential()
    model.add(Dense(100, activation="relu",    kernel_initializer="normal", input_dim=60))
    model.add(Dense(70,  activation="relu",    kernel_initializer="normal"))
    model.add(Dense(1 ,  activation="sigmoid", kernel_initializer="normal"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    
    return model

In [26]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=checking_overfiting, epochs=60, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=60, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Result: 87.22% (16.21%)


In [29]:
def tunning_the_model():
    model = Sequential()
    model.add(Dense(70, activation="relu",    kernel_initializer="normal", input_dim=60))
    model.add(Dense(1,  activation="sigmoid", kernel_initializer="normal"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [30]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tunning_the_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Result: 85.54% (5.72%)


In [42]:
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model
from keras.models import Model

Inputs = Input(shape=(60,))
x = Dense(60, activation='relu')(Inputs)
x = Dense(60, activation='relu')(x)
x = Dense(60, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=Inputs, outputs=output)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        (None, 60)                0         
_________________________________________________________________
dense_391 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_392 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_393 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_394 (Dense)            (None, 1)                 61        
Total params: 11,041
Trainable params: 11,041
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
import tensorflow as tf
from keras.callbacks import History
import keras
class MyModel(tf.keras.Model):
    
        def __init__(self):
            super(MyModel, self).__init__()
            self.dense1= Dense(70, activation='relu')
            self.dense2 = Dense(1, activation='sigmoid')

        def call(self, inputs):
            x = self.dense1(inputs)
            return self.dense2(x)
                        
                    
model = MyModel()

In [46]:
dataframe = shuffle(dataframe)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# label encoding
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

k=10
num_val_samples = len(X)//k
num_epochs = 100

all_acc = numpy.array([])
# all_err = np.array([]

In [47]:
for i in range(k):
    print('Processing K-Fold #', i+1)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = numpy.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = numpy.concatenate(
        [encoded_y[:i * num_val_samples],
         encoded_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = functional_model()
    history = model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    
    results = model.evaluate(val_data, val_targets, verbose=0)
    
#     all_err = np.append(all_err, results[0])
    all_acc = numpy.append(all_acc, results[1])
    
print("+--------------------+")    
print("Custom K-Folds: Accuracy %.2f%%, Error (%.2f%%)" % (all_acc.mean()*100, all_acc.std()*100))

Processing K-Fold # 1
Processing K-Fold # 2
Processing K-Fold # 3
Processing K-Fold # 4
Processing K-Fold # 5
Processing K-Fold # 6
Processing K-Fold # 7
Processing K-Fold # 8
Processing K-Fold # 9
Processing K-Fold # 10
+--------------------+
Custom K-Folds: Accuracy 80.50%, Error (7.89%)
